1. 提取出xs ys

In [ ]:
        xs_last = xs_o[:, -1, :]  
        # # 取出最后一列
        # xs2 = torch.tensor([item[0] for item in xs_o[:, -1]])
        # xs = torch.cat((xs1, xs2), dim=1)

        task = task_sampler(**task_sampler_args)

        ys = task.evaluate(xs_last) # ground truth 变成one-hot向量
        
        xs = xs_o.clone()
        xs[:, -1, point_dims:] = 0 
        print("Shape of xs:", xs.shape)
        print("Shape of ys:", ys.shape)

这是xs ys合在一起的代码

In [ ]:
    def sample_xs(self, n_points, b_size, n_dims_truncated=None, seeds=None):
        assert self.n_dims == 2  # 二维点集
        x_s = []
        
        for _ in range(b_size):
            points, label, point_type = generate_points(n_points, self.n_cluster, seed=seeds)  # 生成点集
            
            # 生成 one-hot 向量
            one_hot_labels = np.zeros((label.size, self.n_cluster))  # 创建一个全零的数组
            one_hot_labels[np.arange(label.size), label] = 1  # 将对应的类别位置设为 1
            
            # 组合点和标签
            labeled_points = []
            for i in range(points.shape[0]):
                point = points[i]  # 取点
                one_hot_label = one_hot_labels[i]  # 取对应的 one-hot 标签
                
                # 组合为一个数组
                combined = np.concatenate((point, one_hot_label))  # 组合为一个数组
                labeled_points.append(combined)  # 直接添加 NumPy 数组

            # 将每个批次的结果转换为一个 NumPy 数组
            x_s.append(np.array(labeled_points))  # 直接添加 NumPy 数组

        # 将所有批次的结果堆叠成一个张量
        x_s = np.array(x_s)  # 转换为 NumPy 数组，形状为 [b_size, n_points, n_dims]
        return torch.tensor(x_s, dtype=torch.float32)  # 确保数据类型为 float32